In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from io import StringIO # String을 파일처럼 다룰 수 있는 도구

In [2]:
csv_data = """A,B,C,D
1.0,2.0,3.0,4.0
5.0,6.0,,8.0
9.0,10.0,11.0,
"""
sio = StringIO(csv_data)
df = pd.read_csv(sio)

In [3]:
df

,A,B,C,D
0,1.0,2.0,3.0,4.0
1,5.0,6.0,NaN,8.0
2,9.0,10.0,11.0,NaN


In [11]:
# missing value 확인

# df.info()
df.isna()
df.isna().sum() # True : 1, False : 0
df.isnull() # isna()와 동일
df.isnull().sum()
df.notna()
df.notnull()

,A,B,C,D
0,True,True,True,True
1,True,True,False,True
2,True,True,True,False


In [18]:
# missing value 처리 1. 삭제 ( 행 또는 열 )

# df.dropna() # NaN이 포함된 모든 행 제거 : axis=0
# df.dropna(axis=1) # NaN이 포함된 모든 열 제거
# df.dropna(how="all") # 모든 데이터가 NaN인 행 제거 (any or all)
# df.dropna(thresh=4) # NaN이 아닌 데이터가 4개 미만인 행 제거
# df.dropna(subset=['C']) # C 컬럼에 NaN이 포함된 행 제거

,A,B,C,D
0,1.0,2.0,3.0,4.0
2,9.0,10.0,11.0,NaN


In [27]:
# missing value 처리 2. 대체
df
c_mean = df["C"].mean()
print(c_mean)
df["C"].fillna(c_mean)
d_median = df['D'].median()
print(d_median)
df['D'].fillna(d_median)

7.0
6.0


0    4.0
1    8.0
2    6.0
Name: D, dtype: float64

In [32]:
# missing value 처리 2. 대체
from sklearn.impute import SimpleImputer

# imputer = SimpleImputer(strategy="mean")
# imputer = SimpleImputer(strategy="median")
# imputer = SimpleImputer(strategy="most_frequent")
imputer = SimpleImputer(strategy="constant", fill_value=100)
imputer.fit(df[['C']]) # strategy 에따라 변환할 값 계산
imputer.transform(df[['C']])

array([[  3.],
       [100.],
       [ 11.]])

In [33]:
df2 = pd.DataFrame([
    ['green', 'M', 10.1, 'class1'],
    ['red', 'L', 13.5, 'class2'],
    ['blue', 'XL', 15.3, 'class1'],
])

df2.columns = ['color', 'size', 'price', 'classlabel']
df2

,color,size,price,classlabel
0,green,M,10.1,class1
1,red,L,13.5,class2
2,blue,XL,15.3,class1


In [34]:
np.unique(df2['classlabel'])

array(['class1', 'class2'], dtype=object)

In [35]:
# 범주형 데이터 -> 수치형 (이산형) 데이터 변환 ( 직접 변환 논리 구현 )
class_mapping = { 'class1' : 0, 'class2' : 1, 'class3' : 2 }
# class_mapping = { label : idx for idx, label in enumerate(np.unique(df2['classlabel'])) }

df2['classlabel'] = df2['classlabel'].map(class_mapping)
df2

,color,size,price,classlabel
0,green,M,10.1,0
1,red,L,13.5,1
2,blue,XL,15.3,0


In [36]:
# 수치형(이산형) 데이터 -> 범주형 데이터로 역변환 ( 직접 변환 논리 구현 )
inv_class_mapping = { v : k for k, v in class_mapping.items() }
# print(inv_class_mapping)
df2['classlabel'] = df2['classlabel'].map(inv_class_mapping)
df2

,color,size,price,classlabel
0,green,M,10.1,class1
1,red,L,13.5,class2
2,blue,XL,15.3,class1


In [40]:
# 범주형 데이터 -> 수치형 (이산형) 데이터 변환 ( 도구 활용 )
from sklearn.preprocessing import LabelEncoder

le = LabelEncoder()
le.fit(df2["classlabel"]) # 변환 규칙 학습
transformed_data = le.transform(df2["classlabel"]) # 학습한 변환 규칙에 따라 변환
print( transformed_data )
df2["classlabel"] = transformed_data
df2

[0 1 0]


,color,size,price,classlabel
0,green,M,10.1,0
1,red,L,13.5,1
2,blue,XL,15.3,0


In [41]:
# 수치형(이산형) 데이터 -> 범주형 데이터로 역변환 ( 도구 활용 )
print( le.inverse_transform(transformed_data) )
df2['classlabel'] = le.inverse_transform(df2['classlabel'])
df2

['class1' 'class2' 'class1']


,color,size,price,classlabel
0,green,M,10.1,class1
1,red,L,13.5,class2
2,blue,XL,15.3,class1
